In [35]:
from pandas.io.parsers import read_csv
import numpy as np

np.random.seed(0) # REMOVE FOR FINAL SOLUTION


In [36]:
#Data Splitted.
def splitData(data):
    shuffled = data.copy()
    np.random.shuffle(shuffled)

    testCount = int(shuffled.shape[0]*1/3)
    trainCount = shuffled.shape[0]-testCount

    trainSet = shuffled[0:trainCount]
    testSet = shuffled[trainCount:]

    return trainSet, testSet

def separateInputOutput(dataset):
    outputData = dataset[:,3] # Output
    inputData = np.concatenate((dataset[:,0:3],dataset[:,4:7],dataset[:,8:10]),axis=1) # Inputs
    return inputData, outputData

#ML ALG
def initializeWeights(x,y):
    N = x.shape[0]
    
def normaliseData(dataArray, normType = "range"):
    normalisedDataList = []

    if normType == "range":
        maxVal = dataArray.max()
        minVal = dataArray.min()
        for val in dataArray:
            normalisedDataList.append((val-minVal)/(maxVal-minVal))
        return np.array(normalisedDataList)

    elif normType == "z":
        mean = dataArray.mean()
        stdDev =  getStandardDeviation(dataArray)
        for val in dataArray:
            normalisedDataList.append((val-mean)/stdDev)
        return np.array(normalisedDataList)

def getStandardDeviation(dataArray):
    n = dataArray.shape[0]
    mean = dataArray.mean()
    sumOfSquareDiffs = 0
    for val in dataArray:
        sumOfSquareDiffs += (val - mean)**2

    variance = sumOfSquareDiffs/(n-1)
    stdDev = variance**(1/2)
    return stdDev


In [37]:
def getUniqueClassCount(listToCount):
    classes = []
    for item in listToCount:
        if item not in classes:
            classes.append(item)

    counts = []
    for itemClass in classes:
        counts.append(listToCount.count(itemClass))
    return counts

def getGiniLeaf(counts):
    total = np.sum(counts)
    giniLeaf = 1
    for count in counts:
        giniLeaf -= (count/total)**2
    return giniLeaf

def getGiniNode(sortedList, threshold):
        lessThan = []
        greaterThan = []

        for i in range(len(sortedList)):
            x = sortedList[i][0]
            # print(x, threshold)
            if x < threshold:
                lessThan.append(sortedList[i][1])
            else:
                greaterThan.append(sortedList[i][1])

        countsLT = getUniqueClassCount(lessThan)
        countsGT = getUniqueClassCount(greaterThan)
        giniLT = getGiniLeaf(countsLT)
        giniGT = getGiniLeaf(countsGT)
        # print(giniLT,giniGT)
        # print(np.sum(countsLT),np.sum(countsGT))
        totalCount = len(sortedList)
        # print(totalCount)
        giniNode = ((np.sum(countsLT)/totalCount)*giniLT) + ((np.sum(countsGT)/totalCount)*giniGT)
        # print(giniNode)
        return giniNode


In [38]:
def getSplitValue(trainIn, trainOut, attributeIndex):
    attribute = trainIn[:, attributeIndex].copy() # Output
    combined = []
    for j,sample in enumerate(attribute):
        combined.append((sample,trainOut[j]))
    attribute.sort()
    sortedList = [tuple for x in attribute for tuple in combined if tuple[0] == x]


    thresholds = []
    for j in range(len(attribute)-1):
        thresholds.append((attribute[j]+attribute[j+1])/2)

    giniNodes = []
    for threshold in thresholds:
        giniNodes.append(getGiniNode(sortedList, threshold))
    # print(giniNodes)
    minNodeGini = min(giniNodes)
    minGiniIndex = giniNodes.index(minNodeGini)
    splitVal = thresholds[minGiniIndex]
    return splitVal, minNodeGini
   

In [65]:
def train(trainIn,trainOut):
    ginis = []
    splitVals = []
    for i in range(trainIn.shape[1]):
        splitVal, minGini = getSplitValue(trainIn,trainOut, i)
        ginis.append(minGini)
        splitVals.append(splitVal)
        # print(splitVal, minGini)
    # print("Root is input data attribute", ginis.index(min(ginis)))
    ltX,lty,gtX,gty = branchData(testIn,testOut,ginis.index(min(ginis)),splitVals[ginis.index(min(ginis))]) #generate root


In [79]:
def branchData(X, y, splitAttributeIndex, splitValue):
    # print(splitAttributeIndex, splitValue)
    # print(X.shape, X[0:2])
    lessThan_X = []
    lessThan_y = []
    greaterThan_X = []
    greaterThan_y = []
    
    for i in range(len(X)):
        if X[i,splitAttributeIndex] < splitValue:
            lessThan_X.append(X[i,:])
            lessThan_y.append(y[i])
        else:
            greaterThan_X.append(X[i,:])
            greaterThan_y.append(y[i])
        
    lessThan_X = np.delete(np.asarray(lessThan_X), splitAttributeIndex, 1)
    lessThan_y = np.asarray(lessThan_y)
    greaterThan_X = np.delete(np.asarray(greaterThan_X), splitAttributeIndex, 1)
    greaterThan_y = np.asarray(greaterThan_y)
    # print(lessThan_X.shape, lessThan_X[0])
    # print(greaterThan_X.shape, greaterThan_X[0])
    return lessThan_X,lessThan_y,greaterThan_X,greaterThan_y
        

In [78]:
#Reading in data
dataframe = read_csv("beer.txt",delim_whitespace=True,header=None)

# Preparing Data
dataset = dataframe.values
trainSet, testSet = splitData(dataset)

testIn, testOut = separateInputOutput(testSet)
trainIn, trainOut = separateInputOutput(trainSet)

N =trainIn.shape[0]
Wi = 1/N

#running ml alg.
train(trainIn,trainOut) #ML ALG RAN HERE


1 0.35662227049999995
(51, 8) [[43.04867257 0.197672705 1.229090909 4.264615385 16.51 13.65547368 16.32
  73.00428571]
 [41.45575221 0.5540157139999999 2.251818182 4.089230768999999 16.97
  10.48668421 12.84 72.56]]
(33, 7) [43.04867257 1.229090909 4.264615385 16.51 13.65547368 16.32 73.00428571]
(18, 7) [41.45575221 2.251818182 4.089230768999999 16.97 10.48668421 12.84 72.56]


In [3]:
class Node:
    def __init__(self,index,value):
        self.left = None
        self.right = None
        self.index = index
        self.value = value #stores value or the classification eg "ale"
        
    def addLeftChild(self,child):
        self.left = child
    
    def getLeftChild(self):
        return self.left

    def addRightChild(self,child):
        self.right = child

    def getRightChild(self):
        return self.right
    
    def printTree(self):
        if self.left:
            self.left.printTree()
        print(self.getData())
        if self.right:
            self.right.printTree()

    def getValue(self):
        return self.value
    
    def getData(self):
        return self.index,self.value
    
    def isLeaf(self):
        if (self.left == None) & (self.right == None):
            return True
        else:
            return False

In [28]:
#Testing Tree
x = [12,11,13]
y = [[9,10,0],[13,11,0],[13,0,14]]
Root = Node(0,x[0])
currentNode =Root
Root.addLeftChild(Node(1,x[1]))
Root.addRightChild(Node(2,x[2]))
currentNode = Root.getLeftChild()
currentNode.addLeftChild(Node(None,"Ale"))
currentNode = Root.getRightChild()
currentNode.addLeftChild(Node(None,"larger"))
currentNode.addRightChild(Node(None,"Wine"))

# Root.printTree()
print(predict(y))


['Ale', 'larger', 'Wine']


In [26]:
#Traversing the tree with input data
def predict(X):
    predictedValues = []
    for i in range(len(X)):
        x = X[i]
        treePointer = Root
        index,value = treePointer.getData()
        while treePointer.isLeaf() == False:
            if (x[index] < value):
                treePointer = treePointer.getLeftChild()
                index,value = treePointer.getData()
            else:
                treePointer = treePointer.getRightChild()
                index,value = treePointer.getData()
        predictedValues.append(treePointer.getValue())
    return predictedValues

    
